# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#get info from csv in weatherpy
cities_df = pd.read_csv("../weatherpy/output_data/city_weather.csv")
cities_df.head()


,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,hobart,41.00,86,147.33,-42.88,75,AU,6.93,1602878272
1,1,nikolskoye,41.00,65,30.79,59.70,90,RU,6.71,1602878273
2,2,coaticook,48.20,100,-71.80,45.13,90,CA,4.70,1602878274
3,3,yumen,41.63,50,97.20,40.28,4,CN,13.51,1602878276
4,4,college,17.01,72,-147.80,64.86,20,US,5.82,1602878277


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Store latitude, longitude and Humidity in locations
locations = cities_df[["Latitude", "Longitude"]]
Humidity = cities_df["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#create new data with specific conditions
ideal_cities = cities_df.loc[(cities_df["Temperature"]>60)\
                             &(cities_df["Temperature"]<90)\
                             &(cities_df["Windspeed"]<20)\
                             &(cities_df["Cloudiness"]<50)].dropna()
ideal_cities

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#create new df *copy to avoid the error message
hotel_df = ideal_cities[["City","Country","Latitude", "Longitude"]].copy()
#add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# find the closest hotels with params using gkey
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    latitude = row['Latitude']
    longitude = row['Longitude']

    # add keyword to params dict
    params['location'] = f'{latitude},{longitude}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['City']} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
#df with set columns and name of hotels
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig